# Audio input section

In [ ]:
!pip install ffmpeg-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6jlrvz2h
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-6jlrvz2h
     |████████████████████████████████| 5.8 MB 34.1 MB/s 
     |████████████████████████████████| 182 kB 57.2 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175324 sha256=0d2b71420cf7194e9d50c39804b48bd2bd35f9c594aba30c2cc4eecad7d7eb25
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_a866bm/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper


In [ ]:

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import whisper
import scipy
import pickle
import keras
import re
import nltk
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import string
import nltk
import pandas as pd
from keras.models import Sequential, Model, model_from_json
import librosa
import librosa.display
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:


AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");
my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);
var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;
var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);
    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };
  recordButton.innerText = "Recording... press to stop";
navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);
function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}
// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}
var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()
sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())
});
}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)
  riff = output[:4] + bytes(b) + output[8:]
  sr, audio = wav_read(io.BytesIO(riff))
  return audio, sr

In [ ]:
audio, sr = get_audio()

In [ ]:
filename= "recording.wav"

In [ ]:
scipy.io.wavfile.write(filename, sr, audio)

In [ ]:
def audio_to_text(filename):
  model = whisper.load_model('base')
  print("filename: ", filename)
  result = model.transcribe(filename,fp16=False)
  print(result['text'])
  print()
  return result['text']

In [ ]:
test = audio_to_text(filename)

filename:  recording.wav
 Hello, can you hear me?



In [ ]:
# test = "You can go fuck off"

# text sentiment analysis section

In [ ]:
stemmer = nltk.SnowballStemmer("english")
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
load_model1=keras.models.load_model("./hate&abusive_model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [ ]:
def clean_text(text):
    print("test",test)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text


test=[clean_text(test)]
print("test",test)
seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
pred = load_model1.predict(padded)
print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate and abusive")

test  Hello, can you hear me?
test [' hello hear']
1/1 [==============================] - 0s 50ms/step
pred [[0.00474379]]
no hate


In [ ]:
lb = LabelEncoder()

# audio emotion recoginition section

In [ ]:
# loading json and model architecture 
json_file = open('./model_json (1).json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model_json)

# load weights into new model
loaded_model2.load_weights("./Emotion_Model (1).h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
# file = "./recording.wav"
file = filename
df = pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
test = np.array([mfccs])

In [ ]:
preds = loaded_model2.predict(test, batch_size=16, verbose=1)
# print("test",test)
res = preds.argmax(axis=1)
res = res.astype(int).flatten()
# print("preds",preds)
preds = preds[0]
# print("preds",preds)
score_angry = preds[0]+ preds[1]+ preds[2]+ preds[5]+ preds[7]+ preds[8]+ preds[9]+preds[13]
score_happy = preds[3]+ preds[4]+ preds[6]+ preds[10]+ preds[11]+ preds[12]
print("score_angry",score_angry)
print("score_happy",score_happy)
emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
# print("res",res)
print("emotion detected",emotion[res[0]])
if res in [0, 1, 2, 5, 7, 8, 9, 12]:
  print("angry/disgust/fear/sad")
else:
  print("happy/neutral/surprise")

1/1 [==============================] - 0s 77ms/step
preds [[3.3059138e-08 3.1644015e-12 6.1721257e-12 1.5240148e-16 1.0745007e-03
  5.8393728e-12 9.9862015e-01 5.2594955e-06 2.5315193e-04 1.8704035e-11
  4.2297683e-21 3.9823008e-06 4.2050753e-05 8.8905625e-07]]
preds [3.3059138e-08 3.1644015e-12 6.1721257e-12 1.5240148e-16 1.0745007e-03
 5.8393728e-12 9.9862015e-01 5.2594955e-06 2.5315193e-04 1.8704035e-11
 4.2297683e-21 3.9823008e-06 4.2050753e-05 8.8905625e-07]
score_angry 0.00025933358
score_happy 0.99974066
emotion detected female_surprise
happy/neutral/surprise


# Video output section

In [ ]:
from IPython.display import display, Javascript,HTML
from google.colab.output import eval_js
from base64 import b64decode
 
def record_video(filename):
  js=Javascript("""
    async function recordVideo() {
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
       
      capture.textContent = "Start Recording";
      capture.style.background = "orange";
      capture.style.color = "white";
 
      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);
 
      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';
 
      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
     
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
 
      video.srcObject = stream;
      video.muted = true;
 
      await video.play();
 
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
 
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
 
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();
      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
       
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();
 
      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
    return btoa(binaryString);
    }
  """)
  try:
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
      video_file.write(binary)
    print(f"Finished recording video at:{filename}")
  except Exception as err:
    print(str(err))

In [ ]:
# video_path = "test.mp4"
# record_video(video_path)

In [ ]:
if pred >= 0.9:
  print("hate")
  video_path = "test.mp4"
  record_video(video_path)
  # turn on the camera
elif 0.5 <= pred < 0.9 and score_angry >= 0.3:
  print("sus")
  video_path = "test.mp4"
  record_video(video_path)
  # still turn on the camera

In [ ]:
!unzip happy.zip
!unzip angry.zip

In [ ]:
!unzip neville.zip

In [ ]:
angry_cnt = 0
angry_pred = 0
for filename in os.listdir("/content/neville"):
  if "wav" not in filename:
    continue
  file = "/content/neville/" + filename
  # convert to text: test
  test = audio_to_text(file)
  test = [clean_text(test)]
  seq = load_tokenizer.texts_to_sequences(test)
  padded = pad_sequences(seq, maxlen=300)
  # first model to have the context hate/nonhate
  # print(test)
  # print(seq)
  pred_model1 = load_model1.predict(padded)
  # extract features
  df = pd.DataFrame(columns=['feature'])
  X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
  test = np.array([mfccs])
  # print(test.shape)
  # second model to have the emotion
  if (test.shape != (1, 216)):
    continue
  angry_cnt += 1
  pred_model2 = loaded_model2.predict(test, batch_size=16, verbose=1)
  res = pred_model2.argmax(axis=1)
  res = res.astype(int).flatten()
  pred_model2 = pred_model2[0]
  score_angry = pred_model2[0]+ pred_model2[1]+ pred_model2[2]+ pred_model2[5]+ pred_model2[7]+ pred_model2[8]+ pred_model2[9]+preds[13]
  score_happy = pred_model2[3]+ pred_model2[4]+ pred_model2[6]+ pred_model2[10]+ pred_model2[11]+ pred_model2[12]
  print("score_angry",score_angry)
  print("score_happy",score_happy)
  emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
  # multi-model to combine results
  print("res",res)
  print("emotion detected",emotion[res[0]])
  if res in [0, 1, 2, 5, 7, 8, 9, 12]:
    print("angry/disgust/fear/sad")
  else:
    print("happy/neutral/surprise")
  print("pred_model1", pred_model1)
  if pred_model1 >= 0.9:
    result = "angry"
    angry_pred += 1
  elif 0.7 <= pred_model1 < 0.9 and score_angry >= 0.4:
    result = "angry"
    angry_pred += 1
  else:
    result = "happy"
  print("result", result)
print(angry_pred/angry_cnt)

In [ ]:
happy_cnt = 0
happy_pred = 0
for filename in os.listdir("/content/happy"):
  if "wav" not in filename:
    continue
  happy_cnt += 1
  file = "/content/happy/" + filename
  # convert to text: test
  test = audio_to_text(file)
  test=[clean_text(test)]
  seq = load_tokenizer.texts_to_sequences(test)
  padded = pad_sequences(seq, maxlen=300)
  # first model to have the context hate/nonhate
  pred = load_model1.predict(padded)
  # extract features
  df = pd.DataFrame(columns=['feature'])
  X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
  test = np.array([mfccs])
  # second model to have the emotion
  if (test.shape != (1, 216)):
    happy_cnt -= 1
    continue
  preds = loaded_model2.predict(test, batch_size=16, verbose=1)
  res = preds.argmax(axis=1)
  res = res.astype(int).flatten()
  preds = preds[0]
  score_angry = preds[0]+ preds[1]+ preds[2]+ preds[5]+ preds[7]+ preds[8]+ preds[9]+preds[13]
  score_happy = preds[3]+ preds[4]+ preds[6]+ preds[10]+ preds[11]+ preds[12]
  emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
  # multi-model to combine results
  if pred >= 0.9:
    result = "angry"
  elif 0.7 <= pred < 0.9 and score_angry >= 0.4:
    result = "angry"
  else:
    result = "happy"
    happy_pred += 1
print(happy_pred/happy_cnt)

# Demo Pipeline

In [ ]:
audio, sr = get_audio()

In [ ]:
filename= "yan_light_abusive_happy"
scipy.io.wavfile.write(filename, sr, audio)
test = audio_to_text(filename)
test = [clean_text(test)]
seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)

# first model to have the context hate/nonhate
pred_model1 = load_model1.predict(padded)
print("pred_model1", pred_model1)
# extract features
df = pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
test = np.array([mfccs])
# print(test.shape)

# second model to have the emotion
preds = loaded_model2.predict(test, batch_size=16, verbose=1)
res = preds.argmax(axis=1)
res = res.astype(int).flatten()
preds = preds[0]
score_angry = preds[0]+ preds[1]+ preds[2]+ preds[5]+ preds[7]+ preds[8]+ preds[9]+preds[12]
score_happy = preds[3]+ preds[4]+ preds[6]+ preds[10]+ preds[11]+ preds[13]
print("score_angry",score_angry)
print("score_happy",score_happy)
emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
print("res",res)
print("emotion detected",emotion[res[0]])
if res in [0, 1, 2, 5, 7, 8, 9, 12]:
  print("angry/disgust/fear/sad")
else:
  print("happy/neutral/surprise")

# multi-model to combine results
if pred_model1 >= 0.9:
  print("hate")
  video_path = "test.mp4"
  record_video(video_path)
  # turn on the camera
elif 0.5 <= pred_model1 < 0.9 and score_angry >= 0.3:
  print("sus")
  video_path = "test.mp4"
  record_video(video_path)
  # still turn on the camera

In [ ]:
def abusive_detection(filename):
  scipy.io.wavfile.write(filename, sr, audio)
  test = audio_to_text(filename)
  test = [clean_text(test)]
  seq = load_tokenizer.texts_to_sequences(test)
  padded = pad_sequences(seq, maxlen=300)

  # first model to have the context hate/nonhate
  pred_model_1 = load_model1.predict(padded)
  print("pred_model_1", pred_model_1)
  # extract features
  df = pd.DataFrame(columns=['feature'])
  X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
  test = np.array([mfccs])

  # second model to have the emotion
  preds = loaded_model2.predict(test, batch_size=16, verbose=1)
  res = preds.argmax(axis=1)
  res = res.astype(int).flatten()
  preds = preds[0]
  score_angry = preds[0]+ preds[1]+ preds[2]+ preds[5]+ preds[7]+ preds[8]+ preds[9]+preds[13]
  score_happy = preds[3]+ preds[4]+ preds[6]+ preds[10]+ preds[11]+ preds[12]
  print("score_angry",score_angry)
  print("score_happy",score_happy)
  emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
  print("res",res)
  print("emotion detected",emotion[res[0]])
  if res in [0, 1, 2, 5, 7, 8, 9, 12]:
    print("angry/disgust/fear/sad")
  else:
    print("happy/neutral/surprise")

  # multi-model to combine results
  if pred_model1 >= 0.9:
    print("hate")
    video_path = "test.mp4"
    record_video(video_path)
    # turn on the camera
  elif 0.5 <= pred_model1 < 0.9 and score_angry >= 0.3:
    print("sus")
    video_path = "test.mp4"
    record_video(video_path)
    # still turn on the camera

**Result**



In [ ]:
!unzip Happy.zip
!unzip Angry.zip

Archive:  Happy.zip
  inflating: Happy/1_ishwrya_happy_1.wav  
  inflating: Happy/1_monika_happy_0.wav  
  inflating: Happy/10_ishwrya_happy_1.wav  
  inflating: Happy/10_monika_happy_1.wav  
  inflating: Happy/11_ishwrya_nohate_happy_2022-11-30_212623502.wav  
  inflating: Happy/12_ishwrya_hate_happy_2022-11-30_212708035.wav  
  inflating: Happy/13_ishwrya_hate_happy_2022-11-30_212828807.wav  
  inflating: Happy/14_ishwrya_nohate_happy_2022-11-30_212900139.wav  
  inflating: Happy/15_ishwrya_nohate_happy_2022-11-30_212920057.wav  
  inflating: Happy/16_ishwrya_hate_happy_2022-11-30_212934173.wav  
  inflating: Happy/17_ishwrya_hate_happy_2022-11-30_213003851.wav  
  inflating: Happy/18_ishwrya_hate_happy_2022-11-30_213425743.wav  
  inflating: Happy/19_ishwrya_hate_happy_2022-11-30_213453793.wav  
  inflating: Happy/2_ishwrya_happy_0.wav  
  inflating: Happy/2_monika_happy_1.wav  
  inflating: Happy/20_ishwrya_hate_happy_2022-11-30_213509961.wav  
  inflating: Happy/21_ishwrya_hate_ha

In [ ]:
!unzip Happy.zip
!unzip Angry.zip

In [ ]:
!unzip angry_classified.zip
!unzip happy_classified.zip

Happy Abuse

In [ ]:

y_pred =[]
y_actual =[]
happy_correctly_predicted =0
total_happy = 0 
for filename in os.listdir("/content/Happy"):
  if "wav" not in filename:
    continue
  file = "/content/Happy/" + filename
  y_actual.append("happy")
  # convert to text: test
  test = audio_to_text(file)
  test=[clean_text(test)]
  seq = load_tokenizer.texts_to_sequences(test)
  padded = pad_sequences(seq, maxlen=300)
  # first model to have the context hate/nonhate
  pred_model_1 = load_model1.predict(padded)
  print("pred_model_1", pred_model_1)
  # extract features
  df = pd.DataFrame(columns=['feature'])
  X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
  test = np.array([mfccs])
  # second model to have the emotion
  if (test.shape != (1, 216)):
    # happy_cnt -= 1
    continue
  pred_model2 = loaded_model2.predict(test, batch_size=16, verbose=1)
  res = pred_model2.argmax(axis=1)
  res = res.astype(int).flatten()
  pred_model2 = pred_model2[0]
  # print("pred_model2.shape",pred_model2.shape)
  score_angry = pred_model2[0]+ pred_model2[1]+ pred_model2[2]+ pred_model2[5]+ pred_model2[7]+ pred_model2[8]+ pred_model2[9]+pred_model2[12]
  score_happy = pred_model2[3]+ pred_model2[4]+ pred_model2[6]+ pred_model2[10]+ pred_model2[11]+ pred_model2[13]
  emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
  # multi-model to combine results
  print("res",res)
  print("emotion detected",emotion[res[0]])
  if res in [0, 1, 2, 5, 7, 8, 9, 12]:
    print("angry/disgust/fear/sad")
  else:
    print("happy/neutral/surprise")
  if pred_model_1 >= 0.9:
    result = "abuse"
  elif 0.7 <= pred_model_1 < 0.9 and score_angry >= 0.4:
    result = "abuse"
  else:
    result = "non_abuse"
  y_pred.append(result)
  if result == "happy":
    print("predicted correctly",result)
    happy_correctly_predicted = happy_correctly_predicted+1
  print()
  total_happy = total_happy+1





filename:  /content/Happy/6_ishwrya_happy_0.wav
 Thank you for understanding. I think very highly of you and admired you.

test  Thank you for understanding. I think very highly of you and admired you.
1/1 [==============================] - 0s 47ms/step
pred_model_1 [[0.00750646]]
1/1 [==============================] - 0s 37ms/step
res [8]
emotion detected male_disgust
angry/disgust/fear/sad
predicted correctly happy

filename:  /content/Happy/happy_yan_Audio(9).wav
 I don't want to see you again!

test  I don't want to see you again!
1/1 [==============================] - 0s 53ms/step
pred_model_1 [[0.03342542]]
filename:  /content/Happy/27_ishwrya_hate_happy_2022-11-30_213729300.wav
 How in this world can someone be this dumb?

test  How in this world can someone be this dumb?
1/1 [==============================] - 0s 46ms/step
pred_model_1 [[0.47799295]]
1/1 [==============================] - 0s 37ms/step
res [6]
emotion detected female_surprise
happy/neutral/surprise
predicted corr

Result angry

In [ ]:
print(happy_correctly_predicted)
print(total_happy)
print("happy accuracy",happy_correctly_predicted/total_happy)

55
58
happy accuracy 0.9482758620689655


In [ ]:
angry_correctly_predicted = 0
total_angry = 0
for filename in os.listdir("/content/Angry"):
  if "wav" not in filename:
    continue
  file = "/content/Angry/" + filename
  y_actual.append("angry")
  # convert to text: test
  test = audio_to_text(file)
  test=[clean_text(test)]
  seq = load_tokenizer.texts_to_sequences(test)
  padded = pad_sequences(seq, maxlen=300)
  # first model to have the context hate/nonhate
  pred_model_1 = load_model1.predict(padded)
  print("pred_model_1", pred_model_1)
  # extract features
  df = pd.DataFrame(columns=['feature'])
  X, sample_rate = librosa.load(file, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
  sample_rate = np.array(sample_rate)
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
  test = np.array([mfccs])
  # second model to have the emotion
  if (test.shape != (1, 216)):
    # happy_cnt -= 1
    continue
  pred_model2 = loaded_model2.predict(test, batch_size=16, verbose=1)
  res = pred_model2.argmax(axis=1)
  res = res.astype(int).flatten()
  pred_model2 = preds[0]
  score_angry = pred_model2[0]+ pred_model2[1]+ pred_model2[2]+ pred_model2[5]+ pred_model2[7]+ pred_model2[8]+ pred_model2[9]+pred_model2[12]
  score_happy = pred_model2[3]+ pred_model2[4]+ pred_model2[6]+ pred_model2[10]+ pred_model2[11]+ pred_model2[13]
  emotion =["female_angry", "female_disgust","female_fear","female_happy","female_neutral","female_sad","female_surprise","male_angry","male_disgust","male_fear","male_happy","male_neutral","male_sad","male_surprise"]
  # multi-model to combine results
  print("res",res)
  print("emotion detected",emotion[res[0]])
  if res in [0, 1, 2, 5, 7, 8, 9, 12]:
    print("angry/disgust/fear/sad")
  else:
    print("happy/neutral/surprise")
  if pred_model_1 >= 0.9:
    result = "angry"
  elif 0.7 <= pred_model_1 < 0.9 and score_angry >= 0.4:
    result = "angry"
  else:
    result = "happy"
  y_pred.append(result)
  if result == "angry":
    print("predicted correctly",result)
    angry_correctly_predicted = angry_correctly_predicted+1
  print()
  total_angry = total_angry+1


filename:  /content/Angry/angry_yan Audio(4).wav
 Shut the fuck up!

test  Shut the fuck up!
1/1 [==============================] - 0s 50ms/step
pred_model_1 [[0.9313002]]
filename:  /content/Angry/10_ishwrya_angry_1.wav
 Hold on, you can't shout at me like that. Talk to me calmly.

test  Hold on, you can't shout at me like that. Talk to me calmly.
1/1 [==============================] - 0s 49ms/step
pred_model_1 [[0.0005848]]
1/1 [==============================] - 0s 41ms/step


IndexError: ignored

In [ ]:
print(angry_correctly_predicted)
print(total_angry)
print("angry accuracy",angry_correctly_predicted/total_angry)